In [97]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib as plt

#### my guess of what the issue is currently the food can't be matched with the libre GL?
perhaps i can approximate with dexcom readings of the food and \
guess on the relative scale of the participants in that dataset and approximate that the same food in this dataset(cgmarcos) \
has the same relative change in glucose given features of the participant (particant_weight_cgm == particant_weight_big_ideas, etc.))

In [21]:
# path of  where the original data is to find a distinct dummy variable
# for pre dia , dia, and non
# C:\Users\najer\OneDrive - UC San Diego\winter_2025\final_project_data\data\cgmacros-a-scientific-dataset-for-personalized-nutrition-and-diet-monitoring-1.0.0

In [4]:
dexcom_equivalent = pd.read_csv("cleaned_data/CGMacros-001.csv")

In [61]:
%ls

 Volume in drive C is Local Disk
 Volume Serial Number is 7C98-7A30

 Directory of C:\Users\najer\OneDrive - UC San Diego\winter_2025\dsc_106\project_4\Final-Project

03/02/2025  12:05 PM    <DIR>          .
02/24/2025  07:40 PM    <DIR>          ..
03/01/2025  09:25 AM                41 .gitignore
03/01/2025  11:29 PM    <DIR>          .ipynb_checkpoints
03/01/2025  11:29 PM    <DIR>          cleaned_data
03/01/2025  11:08 PM    <DIR>          data
03/01/2025  09:25 AM             2,709 data_cleaning.py
02/28/2025  11:58 AM    <DIR>          eda
03/02/2025  12:05 PM            75,902 felix_refactor_dexcom.ipynb
03/01/2025  09:25 AM               458 global.js
03/01/2025  09:25 AM               270 index.html
03/01/2025  09:25 AM               190 README.md
03/02/2025  12:00 PM               236 TODO.md
               7 File(s)         79,806 bytes
               6 Dir(s)  351,498,616,832 bytes free


In [59]:
dexcom_equivalent.head()

,Timestamp,Libre GL,Dexcom GL,HR,Calories (Activity),METs,Meal Type,Calories,Carbs,Protein,Fat,Fiber,Amount Consumed
0,2020-05-01 10:30:00,84.000000,NaN,56.0,1.0484,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-05-01 10:31:00,84.133333,NaN,56.0,1.0484,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-05-01 10:32:00,84.266667,NaN,57.0,1.0484,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-05-01 10:33:00,84.400000,NaN,54.0,1.0484,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-05-01 10:34:00,84.533333,NaN,55.0,1.0484,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
# (dexcom_equivalent["Timestamp"][0]).month
# i think the script is currently missing dropping the 11th day and making timestampe to_datetime?

AttributeError: 'str' object has no attribute 'month'

In [18]:
(dexcom_equivalent["Libre GL"]).value_counts(dropna=False).sum()

np.int64(14730)

In [19]:
(dexcom_equivalent["Dexcom GL"]).value_counts(dropna=False).sum()

np.int64(14730)

#### i think it be fine if we relinquish few hundred points per participant unless we do a prediction of the free space available in the groups
_____

In [22]:
%pwd

'C:\\Users\\najer\\OneDrive - UC San Diego\\winter_2025\\dsc_106\\project_4\\Final-Project'

In [23]:
%ls

 Volume in drive C is Local Disk
 Volume Serial Number is 7C98-7A30

 Directory of C:\Users\najer\OneDrive - UC San Diego\winter_2025\dsc_106\project_4\Final-Project

03/02/2025  10:48 AM    <DIR>          .
02/24/2025  07:40 PM    <DIR>          ..
03/01/2025  09:25 AM                41 .gitignore
03/01/2025  11:29 PM    <DIR>          .ipynb_checkpoints
03/01/2025  11:29 PM    <DIR>          cleaned_data
03/01/2025  11:08 PM    <DIR>          data
03/01/2025  09:25 AM             2,709 data_cleaning.py
02/28/2025  11:58 AM    <DIR>          eda
03/02/2025  10:48 AM             3,220 felix_refactor_dexcom.ipynb
03/01/2025  09:25 AM               458 global.js
03/01/2025  09:25 AM               270 index.html
03/01/2025  09:25 AM               190 README.md
               6 File(s)          6,888 bytes
               6 Dir(s)  351,614,066,688 bytes free


#### we can make groups of  A1c (this is how pre dia, dia, non is declrared)




| normal      | Prediabetes     | diabetes |
| ------------- | ------------- | ------- |
| below 5.7% | : 5.7% to 6.4% | Diabetes: 6.5% or above | 
| *units in A1c*

In [48]:
filtering_participants = pd.read_csv("cleaned_data/bio.csv")

In [53]:
filtering_participants.head()

,PID,Age,Gender,BMI,Body weight,Height,Self-identifid-Race,A1c PDL (Lab),Fasting GLU - PDL (Lab),Insulin,...,HDL,Non HDL,LDL (Cal),VLDL (Cal),Cho/HDL Ratio,Collection time PDL (Lab),Starting GLU,Normal,Prediabetes,Diabetes
0,1,27,M,22.265239,133.8,65.0,Hispanic/Latino,5.4,91,2.5,...,74,142,130,13,2.9,11:06:00 AM,81.000000,1,0,0
1,2,49,F,30.946742,169.2,62.0,Hispanic/Latino,5.5,93,14.8,...,91,90,78,12,2.0,7:38:00 AM,98.000000,1,0,0
2,3,59,F,26.948690,157.0,64.0,Hispanic/Latino,6.5,118,17.4,...,74,116,90,31,2.6,7:25:00 AM,127.666667,0,0,1
3,4,33,F,42.384279,262.6,66.0,Hispanic/Latino,5.5,105,19.4,...,46,221,164,60,5.8,7:20:00 AM,103.000000,1,0,0
4,5,51,F,30.957534,172.0,62.5,Hispanic/Latino,6.6,144,12.9,...,38,231,157,78,7.1,7:45:00 AM,161.333333,0,0,1


In [44]:
filtering_participants["A1c PDL (Lab)"]

0     5.4
1     5.5
2     6.5
3     5.5
4     6.6
5     5.2
6     5.9
7     5.8
8     5.7
9     5.7
10    5.7
11    7.1
12    6.2
13    7.1
14    5.4
15    5.8
16    5.0
17    5.4
18    5.6
19    6.4
20    5.3
21    5.9
22    6.2
23    5.9
24    4.7
25    7.0
26    6.1
27    7.6
28    5.1
29    5.3
30    5.6
31    4.6
32    8.5
33    6.9
34    7.2
35    8.3
36    6.3
37    7.1
38    6.0
39    6.0
40    6.1
41    7.4
42    6.9
43    4.8
44    7.2
Name: A1c PDL (Lab), dtype: float64

In [50]:


def bin_a1c(df):
    df['Normal'] = np.where(df['A1c PDL (Lab)'] < 5.7, 1, 0)
    df['Prediabetes'] = np.where((df['A1c PDL (Lab)'] >= 5.7) & (df['A1c PDL (Lab)'] < 6.5), 1, 0)
    df['Diabetes'] = np.where(df['A1c PDL (Lab)'] >= 6.5, 1, 0)
    return df






In [51]:
participants_group = bin_a1c(filtering_participants)
participants_group


,PID,Age,Gender,BMI,Body weight,Height,Self-identifid-Race,A1c PDL (Lab),Fasting GLU - PDL (Lab),Insulin,...,HDL,Non HDL,LDL (Cal),VLDL (Cal),Cho/HDL Ratio,Collection time PDL (Lab),Starting GLU,Normal,Prediabetes,Diabetes
0,1,27,M,22.265239,133.8,65.00,Hispanic/Latino,5.4,91,2.5,...,74,142,130,13,2.9,11:06:00 AM,81.000000,1,0,0
1,2,49,F,30.946742,169.2,62.00,Hispanic/Latino,5.5,93,14.8,...,91,90,78,12,2.0,7:38:00 AM,98.000000,1,0,0
2,3,59,F,26.948690,157.0,64.00,Hispanic/Latino,6.5,118,17.4,...,74,116,90,31,2.6,7:25:00 AM,127.666667,0,0,1
3,4,33,F,42.384279,262.6,66.00,Hispanic/Latino,5.5,105,19.4,...,46,221,164,60,5.8,7:20:00 AM,103.000000,1,0,0
4,5,51,F,30.957534,172.0,62.50,Hispanic/Latino,6.6,144,12.9,...,38,231,157,78,7.1,7:45:00 AM,161.333333,0,0,1
5,6,51,F,29.303451,197.0,68.75,White,5.2,96,6.4,...,72,131,118,15,2.8,7:45:00 AM,88.333333,1,0,0
6,7,66,F,27.070327,199.6,72.00,Hispanic/Latino,5.9,108,15.9,...,43,85,67,18,3.0,8:00:00 AM,122.333333,0,1,0
7,8,54,M,39.945440,218.4,62.00,Hispanic/Latino,5.8,112,17.7,...,60,120,95,29,3.0,8:55:00 AM,120.000000,0,1,0
8,9,34,F,37.001506,183.2,59.00,Hispanic/Latino,5.7,122,25.7,...,40,162,108,62,5.1,7:32:00 AM,114.333333,0,1,0
9,10,54,F,35.811892,195.8,62.00,Hispanic/Latino,5.7,100,15.3,...,55,128,110,20,3.3,7:44:00 AM,122.666667,0,1,0


#### im using dummy variables since we'll eventually need to use decision trees when the quiz is implemented

In [65]:
participants_group.to_csv("bio_mutated.csv")

In [67]:
%ls

 Volume in drive C is Local Disk
 Volume Serial Number is 7C98-7A30

 Directory of C:\Users\najer\OneDrive - UC San Diego\winter_2025\dsc_106\project_4\Final-Project

03/02/2025  09:03 PM    <DIR>          .
02/24/2025  07:40 PM    <DIR>          ..
03/02/2025  02:41 PM                59 .gitignore
03/01/2025  11:29 PM    <DIR>          .ipynb_checkpoints
03/02/2025  09:03 PM             5,550 bio_mutated.csv
03/01/2025  11:29 PM    <DIR>          cleaned_data
03/01/2025  11:08 PM    <DIR>          data
03/01/2025  09:25 AM             2,709 data_cleaning.py
02/28/2025  11:58 AM    <DIR>          eda
03/02/2025  09:02 PM            74,472 felix_refactor_dexcom.ipynb
03/01/2025  09:25 AM               458 global.js
03/01/2025  09:25 AM               270 index.html
03/01/2025  09:25 AM               190 README.md
03/02/2025  06:25 PM               448 TODO.md
               8 File(s)         84,156 bytes
               6 Dir(s)  351,398,400,000 bytes free


_______

In [5]:
pd.read_csv("data/CGMacros/CGMacros-001/CGMacros-001.csv")

,Unnamed: 0,Timestamp,Libre GL,Dexcom GL,HR,Calories (Activity),METs,Meal Type,Calories,Carbs,Protein,Fat,Fiber,Amount Consumed,Image path
0,0,2020-05-01 10:30:00,84.000000,NaN,56.0,1.04840,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2020-05-01 10:31:00,84.133333,NaN,56.0,1.04840,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2020-05-01 10:32:00,84.266667,NaN,57.0,1.04840,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2020-05-01 10:33:00,84.400000,NaN,54.0,1.04840,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2020-05-01 10:34:00,84.533333,NaN,55.0,1.04840,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14725,14725,2020-05-11 15:55:00,81.666667,NaN,78.0,2.93552,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14726,14726,2020-05-11 15:56:00,81.733333,NaN,79.0,3.14520,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14727,14727,2020-05-11 15:57:00,81.800000,NaN,80.0,3.56456,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14728,14728,2020-05-11 15:58:00,81.866667,NaN,81.0,3.56456,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Missing files: [24, 25, 37, 40]

In [10]:
def find_missing_files(directory):
    missing_files = []
    for i in range(1, 50):
        filename = f"CGMacros-{i:03d}.csv"
        filepath = os.path.join(directory, f"CGMacros-{i:03d}", filename)
        if not os.path.exists(filepath):
            missing_files.append(i)
    return missing_files

directory = "data/CGMacros"
missing_files = find_missing_files(directory)
if missing_files:
    print(f"Missing files: {missing_files}")
else:
    print("All files are present")

Missing files: [24, 25, 37, 40]


In [11]:
excluded_participants = [24, 25, 37, 40]
filenames = [f"data/CGMacros/CGMacros-{i:03d}/CGMacros-{i:03d}.csv" for i in range(1, 50) if i not in excluded_participants]
dataframes = [pd.read_csv(filename) for filename in filenames]

In [53]:
dataframes[0]

,Timestamp,Libre GL,Dexcom GL,HR,Calories (Activity),METs,Meal Type,Calories,Carbs,Protein,Fat,Fiber,Amount Consumed
0,2020-05-01 10:30:00,84.000000,NaN,56.0,1.04840,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-05-01 10:31:00,84.133333,NaN,56.0,1.04840,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-05-01 10:32:00,84.266667,NaN,57.0,1.04840,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-05-01 10:33:00,84.400000,NaN,54.0,1.04840,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-05-01 10:34:00,84.533333,NaN,55.0,1.04840,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14725,2020-05-11 15:55:00,81.666667,NaN,78.0,2.93552,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14726,2020-05-11 15:56:00,81.733333,NaN,79.0,3.14520,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14727,2020-05-11 15:57:00,81.800000,NaN,80.0,3.56456,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14728,2020-05-11 15:58:00,81.866667,NaN,81.0,3.56456,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
def check_missing_values(dataframes):
    columns_to_check = ['Calories', 'Carbs', 'Protein', 'Fat', 'Fiber']
    valid_dataframes = []
    for i, df in enumerate(dataframes):
        for column in columns_to_check:
            if df[column].isna().sum() != len(df):
                valid_dataframes.append(i)
                break
    return valid_dataframes

In [21]:
check_missing_values(dataframes) # find the quantity of data actually present in these dataframes i.e. its 2 meals why bother

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44]

In [22]:
def check_missing_values(dataframes):
    columns_to_check = ['Calories', 'Carbs', 'Protein', 'Fat', 'Fiber']
    valid_dataframes = {}
    for i, df in enumerate(dataframes):
        missing_values = {column: df[column].isna().sum() for column in columns_to_check}
        valid_dataframes[i] = missing_values
    return valid_dataframes

In [23]:
check_missing_values(dataframes)

{0: {'Calories': np.int64(14687),
  'Carbs': np.int64(14687),
  'Protein': np.int64(14687),
  'Fat': np.int64(14687),
  'Fiber': np.int64(14688)},
 1: {'Calories': np.int64(16988),
  'Carbs': np.int64(16988),
  'Protein': np.int64(16988),
  'Fat': np.int64(16988),
  'Fiber': np.int64(16988)},
 2: {'Calories': np.int64(14530),
  'Carbs': np.int64(14530),
  'Protein': np.int64(14530),
  'Fat': np.int64(14530),
  'Fiber': np.int64(14530)},
 3: {'Calories': np.int64(14213),
  'Carbs': np.int64(14213),
  'Protein': np.int64(14213),
  'Fat': np.int64(14213),
  'Fiber': np.int64(14213)},
 4: {'Calories': np.int64(14414),
  'Carbs': np.int64(14414),
  'Protein': np.int64(14414),
  'Fat': np.int64(14414),
  'Fiber': np.int64(14414)},
 5: {'Calories': np.int64(14422),
  'Carbs': np.int64(14422),
  'Protein': np.int64(14422),
  'Fat': np.int64(14422),
  'Fiber': np.int64(14422)},
 6: {'Calories': np.int64(5638),
  'Carbs': np.int64(5638),
  'Protein': np.int64(5638),
  'Fat': np.int64(5638),
  'F

In [24]:
# only participant 6 might even have data directly inputted

___
#### im going to conclusively use the big ideas dataset to approximate ?!?!?!
#### maybe use image recognition to at least find the food?!?!?

## cant use dexcom at all cant use marcos either only have jpgs for (maybe use the xgboost model the paper)
## image recognition and calories

In [26]:
%ls

 Volume in drive C is Local Disk
 Volume Serial Number is 7C98-7A30

 Directory of C:\Users\najer\OneDrive - UC San Diego\winter_2025\dsc_106\project_4\Final-Project

03/03/2025  04:13 PM    <DIR>          .
02/24/2025  07:40 PM    <DIR>          ..
03/02/2025  10:51 PM                59 .gitignore
03/03/2025  03:57 PM    <DIR>          .ipynb_checkpoints
03/01/2025  11:29 PM    <DIR>          cleaned_data
03/03/2025  02:02 PM    <DIR>          data
03/01/2025  09:25 AM             2,709 data_cleaning.py
03/02/2025  10:51 PM             4,813 data_transform.py
02/28/2025  11:58 AM    <DIR>          eda
03/03/2025  04:13 PM           101,938 felix_find_macros.ipynb
03/03/2025  01:35 PM             1,882 global.js
03/03/2025  01:37 PM               889 index.html
03/01/2025  09:25 AM               190 README.md
03/03/2025  01:36 PM                 0 styles.css
03/03/2025  02:04 PM               544 TODO.md
               9 File(s)        113,024 bytes
               6 Dir(s)  344,410,370

In [29]:
excluded_participants = [24, 25, 37, 40]
filenames = [f"cleaned_data/CGMacros-{i:03d}.csv" for i in range(1, 50) if i not in excluded_participants]
dataframes = [pd.read_csv(filename) for filename in filenames]

In [33]:
participant_groups = pd.read_csv("data/CGMacros/bio_mutated.csv")

In [34]:
participant_groups

,Unnamed: 0,PID,Age,Gender,BMI,Body weight,Height,Self-identifid-Race,A1c PDL (Lab),Fasting GLU - PDL (Lab),...,HDL,Non HDL,LDL (Cal),VLDL (Cal),Cho/HDL Ratio,Collection time PDL (Lab),Starting GLU,Normal,Prediabetes,Diabetes
0,0,1,27,M,22.265239,133.8,65.00,Hispanic/Latino,5.4,91,...,74,142,130,13,2.9,11:06:00 AM,81.000000,1,0,0
1,1,2,49,F,30.946742,169.2,62.00,Hispanic/Latino,5.5,93,...,91,90,78,12,2.0,7:38:00 AM,98.000000,1,0,0
2,2,3,59,F,26.948690,157.0,64.00,Hispanic/Latino,6.5,118,...,74,116,90,31,2.6,7:25:00 AM,127.666667,0,0,1
3,3,4,33,F,42.384279,262.6,66.00,Hispanic/Latino,5.5,105,...,46,221,164,60,5.8,7:20:00 AM,103.000000,1,0,0
4,4,5,51,F,30.957534,172.0,62.50,Hispanic/Latino,6.6,144,...,38,231,157,78,7.1,7:45:00 AM,161.333333,0,0,1
5,5,6,51,F,29.303451,197.0,68.75,White,5.2,96,...,72,131,118,15,2.8,7:45:00 AM,88.333333,1,0,0
6,6,7,66,F,27.070327,199.6,72.00,Hispanic/Latino,5.9,108,...,43,85,67,18,3.0,8:00:00 AM,122.333333,0,1,0
7,7,8,54,M,39.945440,218.4,62.00,Hispanic/Latino,5.8,112,...,60,120,95,29,3.0,8:55:00 AM,120.000000,0,1,0
8,8,9,34,F,37.001506,183.2,59.00,Hispanic/Latino,5.7,122,...,40,162,108,62,5.1,7:32:00 AM,114.333333,0,1,0
9,9,10,54,F,35.811892,195.8,62.00,Hispanic/Latino,5.7,100,...,55,128,110,20,3.3,7:44:00 AM,122.666667,0,1,0


In [45]:
indices = {column: np.where(participant_groups[column] == 1)[0] for column in ['Normal', 'Prediabetes', 'Diabetes']}

for key, value in indices.items():
    print(f"Indices of 1's in column '{key}': {value}")

Indices of 1's in column 'Normal': [ 0  1  3  5 14 16 17 18 20 24 28 29 30 31 43]
Indices of 1's in column 'Prediabetes': [ 6  7  8  9 10 12 15 19 21 22 23 26 36 38 39 40]
Indices of 1's in column 'Diabetes': [ 2  4 11 13 25 27 32 33 34 35 37 41 42 44]


In [46]:
indices

{'Normal': array([ 0,  1,  3,  5, 14, 16, 17, 18, 20, 24, 28, 29, 30, 31, 43]),
 'Prediabetes': array([ 6,  7,  8,  9, 10, 12, 15, 19, 21, 22, 23, 26, 36, 38, 39, 40]),
 'Diabetes': array([ 2,  4, 11, 13, 25, 27, 32, 33, 34, 35, 37, 41, 42, 44])}

In [41]:
np.array(missing_files) - 1

array([23, 24, 36, 39])

In [47]:
remove_indices = [23, 24, 36, 39]

# remove entry pairs with equal to missing participants
for key, value in list(indices.items()):
    indices[key] = np.array([x for x in value if x not in remove_indices])

In [48]:
indices

{'Normal': array([ 0,  1,  3,  5, 14, 16, 17, 18, 20, 28, 29, 30, 31, 43]),
 'Prediabetes': array([ 6,  7,  8,  9, 10, 12, 15, 19, 21, 22, 26, 38, 40]),
 'Diabetes': array([ 2,  4, 11, 13, 25, 27, 32, 33, 34, 35, 37, 41, 42, 44])}

In [49]:
participant_groups

,Unnamed: 0,PID,Age,Gender,BMI,Body weight,Height,Self-identifid-Race,A1c PDL (Lab),Fasting GLU - PDL (Lab),...,HDL,Non HDL,LDL (Cal),VLDL (Cal),Cho/HDL Ratio,Collection time PDL (Lab),Starting GLU,Normal,Prediabetes,Diabetes
0,0,1,27,M,22.265239,133.8,65.00,Hispanic/Latino,5.4,91,...,74,142,130,13,2.9,11:06:00 AM,81.000000,1,0,0
1,1,2,49,F,30.946742,169.2,62.00,Hispanic/Latino,5.5,93,...,91,90,78,12,2.0,7:38:00 AM,98.000000,1,0,0
2,2,3,59,F,26.948690,157.0,64.00,Hispanic/Latino,6.5,118,...,74,116,90,31,2.6,7:25:00 AM,127.666667,0,0,1
3,3,4,33,F,42.384279,262.6,66.00,Hispanic/Latino,5.5,105,...,46,221,164,60,5.8,7:20:00 AM,103.000000,1,0,0
4,4,5,51,F,30.957534,172.0,62.50,Hispanic/Latino,6.6,144,...,38,231,157,78,7.1,7:45:00 AM,161.333333,0,0,1
5,5,6,51,F,29.303451,197.0,68.75,White,5.2,96,...,72,131,118,15,2.8,7:45:00 AM,88.333333,1,0,0
6,6,7,66,F,27.070327,199.6,72.00,Hispanic/Latino,5.9,108,...,43,85,67,18,3.0,8:00:00 AM,122.333333,0,1,0
7,7,8,54,M,39.945440,218.4,62.00,Hispanic/Latino,5.8,112,...,60,120,95,29,3.0,8:55:00 AM,120.000000,0,1,0
8,8,9,34,F,37.001506,183.2,59.00,Hispanic/Latino,5.7,122,...,40,162,108,62,5.1,7:32:00 AM,114.333333,0,1,0
9,9,10,54,F,35.811892,195.8,62.00,Hispanic/Latino,5.7,100,...,55,128,110,20,3.3,7:44:00 AM,122.666667,0,1,0


In [50]:
dataframes_normal = [dataframes[i] for i in indices['Normal']]
dataframes_prediabetes = [dataframes[i] for i in indices['Prediabetes']]
dataframes_diabetes = [dataframes[i] for i in indices['Diabetes']]

In [52]:
dataframes_normal[0]

,Timestamp,Libre GL,Dexcom GL,HR,Calories (Activity),METs,Meal Type,Calories,Carbs,Protein,Fat,Fiber,Amount Consumed
0,2020-05-01 10:30:00,84.000000,NaN,56.0,1.04840,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-05-01 10:31:00,84.133333,NaN,56.0,1.04840,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-05-01 10:32:00,84.266667,NaN,57.0,1.04840,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-05-01 10:33:00,84.400000,NaN,54.0,1.04840,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-05-01 10:34:00,84.533333,NaN,55.0,1.04840,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14725,2020-05-11 15:55:00,81.666667,NaN,78.0,2.93552,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14726,2020-05-11 15:56:00,81.733333,NaN,79.0,3.14520,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14727,2020-05-11 15:57:00,81.800000,NaN,80.0,3.56456,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14728,2020-05-11 15:58:00,81.866667,NaN,81.0,3.56456,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
len(dataframes_normal)

14

___
TODO normalize across the groups for now graph absolute GL across the groups of dataframes

In [62]:
normal_gl_df = pd.concat([df['Libre GL'] for df in dataframes_normal], axis=1)
pre_gl_df = pd.concat([df['Libre GL'] for df in dataframes_prediabetes], axis=1)
dia_gl_df = pd.concat([df['Libre GL'] for df in dataframes_diabetes], axis=1)


In [64]:
dia_gl_df

,Libre GL,Libre GL,Libre GL,Libre GL,Libre GL,Libre GL,Libre GL,Libre GL,Libre GL,Libre GL,Libre GL,Libre GL,Libre GL,Libre GL
0,155.0,109.0,198.000000,179.000000,102.000000,167.000000,274.000000,149.000000,176.0,211.000000,160.000000,249.000000,159.000000,154.000000
1,155.0,110.0,199.933333,179.733333,101.533333,167.533333,272.733333,149.133333,176.4,210.866667,158.666667,249.666667,157.866667,157.066667
2,155.0,111.0,201.866667,180.466667,101.066667,168.066667,271.466667,149.266667,176.8,210.733333,157.333333,250.333333,156.733333,160.133333
3,155.0,112.0,203.800000,181.200000,100.600000,168.600000,270.200000,149.400000,177.2,210.600000,156.000000,251.000000,155.600000,163.200000
4,155.0,113.0,205.733333,181.933333,100.133333,169.133333,268.933333,149.533333,177.6,210.466667,154.666667,251.666667,154.466667,166.266667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18730,NaN,NaN,NaN,NaN,114.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18731,NaN,NaN,NaN,NaN,114.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18732,NaN,NaN,NaN,NaN,114.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18733,NaN,NaN,NaN,NaN,114.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
populated_normal = normal_gl_df.dropna(how='any')

# calculate the average of the row
temp_df_0 = (populated_normal.sum(axis=1)) / len(populated_normal.columns)

temp_df_0 = pd.DataFrame(temp_df_0)

In [107]:
temp_df_0

,0
0,83.428571
1,83.252381
2,83.076190
3,82.900000
4,82.723810
...,...
13720,94.976190
13721,95.180952
13722,95.385714
13723,95.590476


In [82]:
populated_pre = pre_gl_df.dropna(how='any')

# calculate the average of the row
temp_df_1 = (populated_pre.sum(axis=1)) / len(populated_pre.columns)

temp_df_1 = pd.DataFrame(temp_df_1)

In [108]:
temp_df_1

,0
0,116.923077
1,117.107692
2,117.292308
3,117.476923
4,117.661538
...,...
5650,96.820513
5651,96.717949
5652,96.615385
5653,96.512821


In [83]:
populated_dia = dia_gl_df.dropna(how='any')

# calculate the average of the row
temp_df_2 = (populated_dia.sum(axis=1)) / len(populated_dia.columns)

temp_df_2 = pd.DataFrame(temp_df_2)

In [86]:
final_df = pd.concat([temp_df_0, temp_df_2,temp_df_1], axis=1)
final_df.columns = ["Normal", "Diabetic","Pre-Diabetic"
                   ]
final_df

,Normal,Diabetic,Pre-Diabetic
0,83.428571,174.428571,116.923077
1,83.252381,174.723810,117.107692
2,83.076190,175.019048,117.292308
3,82.900000,175.314286,117.476923
4,82.723810,175.609524,117.661538
...,...,...,...
14395,NaN,174.809524,NaN
14396,NaN,174.790476,NaN
14397,NaN,174.771429,NaN
14398,NaN,174.752381,NaN


In [93]:
time_truncated = pd.DataFrame(dataframes[0]['Timestamp'].values[:len(final_df)])

In [96]:
final_df["Timestamp"] = time_truncated
final_df

,Normal,Diabetic,Pre-Diabetic,Timestamp
0,83.428571,174.428571,116.923077,2020-05-01 10:30:00
1,83.252381,174.723810,117.107692,2020-05-01 10:31:00
2,83.076190,175.019048,117.292308,2020-05-01 10:32:00
3,82.900000,175.314286,117.476923,2020-05-01 10:33:00
4,82.723810,175.609524,117.661538,2020-05-01 10:34:00
...,...,...,...,...
14395,NaN,174.809524,NaN,2020-05-11 10:25:00
14396,NaN,174.790476,NaN,2020-05-11 10:26:00
14397,NaN,174.771429,NaN,2020-05-11 10:27:00
14398,NaN,174.752381,NaN,2020-05-11 10:28:00


In [104]:
final_df.to_csv("absolute_gl_values_grouped_h1Ac.csv")

In [103]:
plt.plot(final_df['Timestamp'], final_df['Normal'], label='Normal')
plt.plot(final_df['Timestamp'], final_df['Pre-Diabetic'], label='Pre-Diabetic')
plt.plot(final_df['Timestamp'], final_df['Diabetic'], label='Diabetic')

plt.xlabel('Timestamp')
plt.ylabel('GL')
plt.title('Absolute GL values of Normal, Pre-Diabetic, and Diabetic')
plt.legend()

plt.show()

AttributeError: module 'matplotlib' has no attribute 'plot'

In [ ]:
# df_melt = final_df.melt(id_vars='Timestamp', value_vars=['Normal', 'Diabetic', 'Pre-Diabetic'])
# # df_melt
# sns.lineplot(x='Timestamp', y='value', hue='variable', data=df_melt)

